In [ ]:
import json
import sqlite3

# Load JSON data
with open("../data/faq_en.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Connect to SQLite database
conn = sqlite3.connect("../data/faq_database.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS faq (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    section_name TEXT,
    section_id TEXT,
    faq_subsection_name TEXT,
    questions TEXT,
    response TEXT,
    embeddings BLOB
)
""")

# Parse and insert data
for section in data["data"]:
    section_name = section["attributes"]["Sekcia"]
    section_id = section["attributes"]["SekciaID"]

    for subsection in section["attributes"].get("Podsekcia_s_otazkami_a_odpovedami", []):
        faq_subsection_name = subsection["Podsekcia"]
        
        for qa in subsection["Otazky_a_odpovede"]:
            question = json.dumps([qa["Otazka"]], ensure_ascii=False)
            response = qa["Odpoved"]

            cursor.execute("""
                INSERT INTO faq (section_name, section_id, faq_subsection_name, questions, response)
                VALUES (?, ?, ?, ?, ?)
            """, (section_name, section_id, faq_subsection_name, question, response))

# Commit and close connection
conn.commit()
conn.close()

print("Data inserted successfully!")


In [ ]:
import ast
def generate_questions(question,response, n=5):
    prompt = f"""
    Your task is to generate {n} questions based on the following text:
    Text: "{response}"
    Keep in mind, that the questions should be relevant to the following example question:
    {question}
    Format your questions as a python list, each question should be in double quotes and separated by a comma.
    Example:
    [
        "Where can I donate blood?",
        "Where are the blood donation centres?",
        "Where to go for blood donation?"
    ]
    The questions should be generated based on the context of the provided text. The questions should be open-ended. 
    The questions should be in the language of the provided text.
    """
    response = litellm.completion(
        model="azure/gpt-4",
        deployment_id="gpt-4-turbo",
        messages=[{ "content": prompt,"role": "user"}])
    generated_questions = response.choices[0].message.content
    print(generated_questions)
    return generated_questions


In [ ]:
chitchat_dict={
    "Som Eryc a moje meno je odvodené od slova erytrocyt, čiže červená krvinka. Snažím sa pomôcť ľuďom, aby sa rýchlejšie zorientovali v téme darovania krvi. Okrem toho viem pomôcť aj s plánovaním termínu darovania krvi.":["Ako sa voláš?","Aké je tvoje meno?","Kto si?","Čo dokážeš?","Čo si zač?","Akú máš funkciu?","Akú máš úlohu?","Akú máš prácu?","Čo robíš?","Prečo sa voláš Eryc?","Prečo sa voláš tak ako sa voláš?","Prečo sa voláš Eryc a nie inak?","Prečo sa voláš Eryc a nie Erik?"],
    "I am Eryc and my name is derived from the word erythrocyte, which means red blood cell. I try to help people to get oriented in the topic of blood donation faster. In addition, I can also help with planning a blood donation date.":["What is your name?","What can you do?","What do you call yourself?","Who are you?","What are you?","What is your function?","What is your role?","What is your job?","What do you do?","Why are you called Eryc?","Why are you called what you are called?","Why are you called Eryc and not something else?","Why are you called Eryc and not Eric?"],
}

import sqlite3
import json
import litellm
import os
import numpy as np
# Set up LiteLLM with Azure OpenAI credentials
litellm.api_key = os.getenv("AZURE_API_KEY")
litellm.api_base = os.getenv("AZURE_API_BASE")


def generate_embeddings(query):
    response = litellm.embedding(
        model="azure/text-embedding-3-large",
        deployment_id="text-embedding-3-large",
        api_version="2023-05-15",
        input=query)
    return np.array(response["data"][0]["embedding"], dtype='float32')

# Connect to SQLite
conn = sqlite3.connect("../data/faq_database.db")
cursor = conn.cursor()

for k,v in chitchat_dict.items():

    # Combine questions and response
    combined_text = " ".join(v) + " " + k

    # Get embedding
    embedding_vector = generate_embeddings(combined_text)
    embedding_blob = embedding_vector.tobytes()
    # Store in the database
    cursor.execute("""
        INSERT INTO faq (section_name, section_id, faq_subsection_name, questions, response, embeddings)
        VALUES (?, ?, ?, ?, ?,?)
    """, ("CHITCHAT", "CHITCHAT", "CHITCHAT",json.dumps(v,ensure_ascii=False) , k, embedding_blob))

# Commit and close
conn.commit()
conn.close()

In [ ]:
import json
import sqlite3

# Load JSON data

# Connect to SQLite database
conn = sqlite3.connect("../data/faq_database.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
ALTER TABLE feedbacks ADD COLUMN TIMESTAMP float;
""")
conn.commit()
conn.close()

In [ ]:

# Connect to SQLite
conn = sqlite3.connect("../data/faq_database.db")
cursor = conn.cursor()
# Retrieve all rows
cursor.execute("SELECT id, questions, response FROM faq")  # Adjust table name if needed
rows = cursor.fetchall()

for row in rows:
    entry_id, questions_json, response_text = row

    # Convert JSON string to list
    base_question = json.loads(questions_json)[0]
    if entry_id > 36:
        # Generate questions
        generated_questions = json.loads(generate_questions(base_question,response_text)) 
        generated_questions = [question.replace("\n","") for question in generated_questions]
        generated_questions.append(base_question)
        generated_questions = json.dumps(generated_questions, ensure_ascii=False)
        
        # Store in the database
        cursor.execute(
            "UPDATE faq SET questions = ? WHERE id = ?",
            (generated_questions, entry_id)
        )
        conn.commit()
    else:
        continue
# Commit and close

conn.close()

In [ ]:
import sqlite3
# Connect to SQLite database
conn = sqlite3.connect("../data/calendar.db")
cursor = conn.cursor()

# Create bookings table
cursor.execute("""
CREATE TABLE IF NOT EXISTS booking (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    email TEXT NOT NULL,
    timestamp INTEGER NOT NULL,
    donation_type TEXT NOT NULL,
    location TEXT NOT NULL,
    success BOOLEAN NOT NULL,
    status TEXT NOT NULL,
    note TEXT
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS slots (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    location TEXT NOT NULL,
    timestamp INTEGER NOT NULL,
    slots_total INTEGER NOT NULL,
    slots_remaining INTEGER NOT NULL
)
""")


conn.commit()
conn.close()

In [ ]:
import sqlite3
conn = sqlite3.connect("../data/calendar.db")
import random
from datetime import datetime, timedelta

cursor = conn.cursor()

# Locations
locations = [
    "Banská Bystrica", "Bratislava - Kramáre", "Bratislava - Ružinov", "Košice",
    "Martin", "Nitra", "Nové Zámky", "Poprad", "Prešov", "Trenčín", "Trnava", "Žilina"
]

# Date range: March 11 to March 24, 2025
start_date = datetime(2025, 5, 1, 7, 0)  # 07:00 AM on March 11
end_date = datetime(2025, 6, 30, 15, 0)  # 03:00 PM on March 24

# Generate time slots (every 15 minutes from 07:00 to 15:00)
time_slots_per_day = [(start_date + timedelta(minutes=30 * i)).timestamp() // 60 for i in range(0, 8 * 4)] 

# Insert data for each location and each day
current_date = start_date
while current_date <= end_date:
    for location in locations:
        for timestamp in time_slots_per_day:
            adjusted_timestamp = int(timestamp + (current_date - start_date).total_seconds() // 60)
            slots_total = 3
            slots_remaining = random.randint(0, 3)  # Random available slots

            cursor.execute("""
                INSERT INTO slots (location, timestamp, slots_total, slots_remaining)
                VALUES (?, ?, ?, ?)
            """, (location, adjusted_timestamp, slots_total, slots_remaining))

    current_date += timedelta(days=1)  # Move to the next day

# Commit and close connection
conn.commit()
conn.close()

print("Dummy data inserted successfully!")


In [ ]:
def format_timestamp(unix_minutes):
    """
    Converts a Unix timestamp in minutes to a formatted time string.
    
    - If the timestamp is today → returns "HH:MM"
    - If the timestamp is another day → returns "DD.MM. HH:MM"

    :param unix_minutes: Unix timestamp in minutes
    :return: Formatted time string
    """
    dt = datetime.fromtimestamp(unix_minutes * 60)  # Convert to datetime
    today = datetime.now().date()

    if dt.date() == today:
        return dt.strftime("%H:%M")  # Format as "HH:MM"
    else:
        return dt.strftime("%d.%m. %H:%M")  # Format as "DD.MM. HH:MM"

timestamp=29055984

formatted_timestamp = format_timestamp(timestamp)
print(formatted_timestamp)

In [ ]:
from ruamel.yaml import YAML
yaml=YAML()
import json
import io
def yaml_dump(data):
    dump = io.StringIO()
    # yaml.round_trip_dump(data, dump)
    yaml.dump(data, dump)
    return dump.getvalue()
with open("data/donation_questionnaire_responses.json", "r", encoding="utf-8") as file:
    data = json.load(file)
with open("../data/slovak_questionnaire_responses.json") as f:
    slovak_data = json.load(f)

responses_out = {}
for name, response in data.items():
    slovak_text = slovak_data[name]
    responses_out[name] = response
    slot = name.split("utter_ask_")[1]
    responses_out[name].append(dict(text=slovak_text,condition=[dict(type="slot", name="slot_lang", value="sk")],
                                    buttons=[
                                        dict(title="Áno",
                                        payload=f"/SetSlots({slot}=yes)",
                                        ),
                                        dict(title="Nie",
                                            payload=f"/SetSlots({slot}=no)",
                                        )]))
print(yaml_dump(dict(responses=responses_out)))

In [ ]:
import sqlite3
# Connect to SQLite database
conn = sqlite3.connect("../data/calendar.db")
cursor = conn.cursor()

# Create table
cursor.execute("""
CREATE TABLE IF NOT EXISTS forms (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    email TEXT not null,
    data TEXT,
    timestamp TEXT,
    expires TEXT,
    form_id TEXT
)
""")
conn.commit()
conn.close()

In [ ]:
from datetime import datetime
ts=28941630
def format_timestamp(unix_minutes):
    """
    Converts a Unix timestamp in minutes to a formatted time string.
    
    - If the timestamp is today → returns "HH:MM"
    - If the timestamp is another day → returns "DD.MM. HH:MM"

    :param unix_minutes: Unix timestamp in minutes
    :return: Formatted time string
    """
    dt = datetime.fromtimestamp(unix_minutes * 60)  # Convert to datetime
    today = datetime.now().date()

    if dt.date() == today:
        return dt.strftime("%H:%M")  # Format as "HH:MM"
    else:
        return dt.strftime("%d.%m.%y %H:%M")  # Format as "DD.MM. HH:MM"
print(format_timestamp(ts))

In [ ]:
import sqlite3

conn = sqlite3.connect('../data/faq_database.db')
cursor = conn.cursor()

cursor.execute("SELECT name, sql FROM sqlite_master WHERE type='table';")
for name, sql in cursor.fetchall():
    print(f"Table: {name}\n{sql}\n")
